<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Подключение-библиотек" data-toc-modified-id="Подключение-библиотек-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Подключение библиотек</a></span></li></ul></li><li><span><a href="#Проект" data-toc-modified-id="Проект-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Проект</a></span></li></ul></div>

## Подключение библиотек

In [7]:
import requests
from IPython.lib import backgroundjobs as bg
from os import path

In [13]:
# https://habr.com/ru/post/276725/
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [14]:
def download_data(from_url, to_file):
    r = requests.get(from_url, stream=True)
    with open(to_file, 'wb') as f:
        total_length = int(r.headers.get('content-length'))
        for chunk in log_progress(r.iter_content(chunk_size=1024), every=100, size=int(total_length/1024) + 1):
            if chunk:
                f.write(chunk)
                f.flush()

In [15]:
def get_url(year, month):
    return "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_{}-{:02d}.csv".format(year, month)

In [16]:
def get_data_file_name(year, month):
    return "{}-{:02d}.csv".format(year, month)

In [17]:
download_data(get_url(2016, 5), get_data_file_name(2016, 5))

In [5]:
jobs = bg.BackgroundJobManager()

In [6]:
for year in range(2016,2020):
    for month in range(1,13):
        jobs.new(download_data, get_url(year, month), path.join('data', get_data_file_name(year, month)))

# Проект